In [217]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import skimage
from skimage.io import imread,imshow
import glob
import pandas as pd
import pickle
import time
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# cd drive/My Drive/BTP

/content/drive/My Drive/BTP


In [199]:
df=pd.read_csv(r'Results/E3D3/1.csv')

In [200]:
with open('Data/imputed_valid_data.pkl', 'rb') as picklefile:
    valid_data=pickle.load(picklefile)

with open('Models/DR/trainedPCA.pkl', 'rb') as picklefile:
    pca=pickle.load(picklefile)

In [191]:
with open('Models/DR/trainedPCA.pkl', 'rb') as picklefile:
    pca=pickle.load(picklefile)

In [201]:
df.head()

,date,unit1,unit2,unit3,unit4,unit5,unit6,unit7,unit8,unit9,...,unit578,unit579,unit580,unit581,unit582,unit583,unit584,unit585,unit586,unit587
0,2020-07-08,37.893044,-6.626562,-14.363065,-2.899816,-6.745531,-4.564622,8.635415,11.501621,-5.259525,...,0.013795,-0.062003,-0.020977,-0.006206,-0.125612,0.071071,-0.036371,-0.160064,-0.007864,0.074677
1,2020-07-09,37.355618,-7.616899,-14.694006,-2.867782,-6.573371,-4.424226,9.087059,11.273763,-4.979779,...,0.019160,-0.069404,-0.024627,-0.007495,-0.127096,0.077193,-0.044566,-0.158863,-0.006555,0.074795
2,2020-07-10,36.788560,-8.494826,-15.200465,-2.829158,-6.362743,-4.255680,9.390262,11.033648,-4.891525,...,0.022288,-0.074822,-0.026745,-0.007124,-0.128734,0.080094,-0.047821,-0.154385,-0.007464,0.073396
3,2020-07-11,36.864944,-8.954903,-15.640271,-2.644531,-6.268304,-4.096462,9.598229,10.758110,-4.770917,...,0.024214,-0.080978,-0.030006,-0.004581,-0.128664,0.086246,-0.049963,-0.147106,-0.006631,0.066907
4,2020-07-12,37.074740,-9.451205,-16.068499,-2.553373,-6.172836,-3.813950,9.689317,10.404470,-4.733233,...,0.025154,-0.088374,-0.030111,-0.001729,-0.129345,0.091305,-0.049953,-0.141751,-0.006511,0.060997


In [202]:
daf=df.drop(['date'],axis='columns')

In [203]:
fpred=pca.inverse_transform(daf.values)

In [205]:
ad=np.reshape(fpred,(len(fpred),256,272,1))

In [206]:
ad.shape  

(88, 256, 272, 1)

In [204]:
fpred.shape

(88, 69632)

In [207]:
for i,date in enumerate(df['date'].values):
    scaler = MinMaxScaler(feature_range=(0, 1))
    img = scaler.fit_transform(abs(fpred[i]).reshape(-1,1))
    img=np.reshape(img,(256,272))
    ad[i]=img.reshape((256,272,1))
    matplotlib.image.imsave('Final Pred/E3D3/'+date+'.jpg', img,cmap='gray')
    time.sleep(0.1)
    # break

In [208]:
valid_data = valid_data[45:-6]

### Score

In [209]:
from skimage.measure import compare_ssim as ssim

def SSIM(y_true, y_pred):
    return tf.reduce_mean(ssim(y_true, y_pred,multichannel=True))

def PSNR(y_true, y_pred):
    return tf.reduce_mean(tf.image.psnr(y_true, y_pred, 1.0))

In [210]:
from sklearn.metrics import mean_absolute_error,r2_score
print(mean_absolute_error(valid_data.reshape((len(valid_data),256*272)),ad.reshape((len(valid_data),256*272))))

0.05473658267395201


In [211]:
ps = tf.image.psnr(valid_data,ad, 1)

In [212]:
ps.numpy().mean()

17.88011

In [213]:
ss=[]
for i in range(0,len(valid_data)):
  # print(i)
  ss.append(ssim(valid_data[i],ad[i],multichannel=True))

In [214]:
sum(ss)/len(ss)

0.8534157710800195